In [2]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
from itertools import combinations
from collections import defaultdict
import sys
import os

# Add packages to path
sys.path.append('/Users/shihcheng/Documents/Work/Python/uwb-localization-mesh/packages')
sys.path.append('/Users/shihcheng/Documents/Work/Python/uwb-localization-mesh')

from packages.datatypes.datatypes import AnchorConfig
from packages.localization_algos.edge_creation.transforms import create_relative_measurement
from packages.localization_algos.edge_creation.anchor_edges import create_anchor_anchor_edges
from packages.localization_algos.pgo.solver import PGOSolver

# Default anchor positions (from datatypes README)
DEFAULT_ANCHOR_POSITIONS = {
    0: np.array([480, 600, 0]),  # top-right
    1: np.array([0, 600, 0]),    # top-left  
    2: np.array([480, 0, 0]),    # bottom-right
    3: np.array([0, 0, 0])       # bottom-left
}

In [3]:
def load_data(csv_path: str) -> List[Dict]:
    """Load and parse the CSV data."""
    data = []
    with open(csv_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                # Parse the filtered_binned_data_json
                filtered_data = json.loads(row['filtered_binned_data_json'])
                row['filtered_data'] = filtered_data
                
                # Parse numeric fields
                row['ground_truth_x'] = float(row['ground_truth_x'])
                row['ground_truth_y'] = float(row['ground_truth_y'])
                row['ground_truth_z'] = float(row['ground_truth_z'])
                row['pgo_x'] = float(row['pgo_x'])
                row['pgo_y'] = float(row['pgo_y'])
                row['pgo_z'] = float(row['pgo_z'])
                
                data.append(row)
            except (json.JSONDecodeError, KeyError, ValueError) as e:
                print(f"Warning: Skipping row due to parsing error: {e}")
                continue
    return data

In [4]:

def mask_measurements(measurements: Dict[str, List[List[float]]], 
                     selected_anchors: List[int]) -> Dict[str, List[List[float]]]:
    """Mask measurements to only include selected anchors."""
    masked = {}
    for anchor_id in selected_anchors:
        anchor_key = str(anchor_id)
        if anchor_key in measurements:
            masked[anchor_key] = measurements[anchor_key]
    return masked

In [5]:

def transform_measurements_to_global(measurements: Dict[str, List[List[float]]], 
                                   phone_node_id: int = 0) -> List[Tuple[str, str, np.ndarray]]:
    """Transform local measurements to global coordinates and create edges."""
    edges = []
    
    for anchor_id_str, vectors in measurements.items():
        anchor_id = int(anchor_id_str)
        for vector in vectors:
            # Convert to numpy array and transform to global coordinates
            local_vector = np.array(vector)
            try:
                from_node, to_node, global_vector = create_relative_measurement(
                    anchor_id, phone_node_id, local_vector
                )
                edges.append((from_node, to_node, global_vector))
            except ValueError as e:
                # Skip invalid measurements
                continue
    
    return edges

In [6]:
def run_pgo_with_masked_measurements(measurements: Dict[str, List[List[float]]], 
                                   selected_anchors: List[int]) -> Optional[np.ndarray]:
    """Run PGO with measurements masked to selected anchors."""
    
    # Mask measurements to only include selected anchors
    masked_measurements = mask_measurements(measurements, selected_anchors)
    
    if not masked_measurements:
        return None
    
    # Transform measurements to global coordinates
    edges = transform_measurements_to_global(masked_measurements)
    
    if not edges:
        return None
    
    # Create anchor config for ALL anchors (positions always available)
    anchor_config = AnchorConfig(positions=DEFAULT_ANCHOR_POSITIONS)
    
    # Add anchor-anchor edges for ALL anchors
    anchor_edges = create_anchor_anchor_edges(anchor_config)
    all_edges = edges + anchor_edges
    
    # Set up nodes (ALL anchors are fixed, phone is unknown)
    nodes = {}
    for anchor_id in DEFAULT_ANCHOR_POSITIONS.keys():
        nodes[f'anchor_{anchor_id}'] = DEFAULT_ANCHOR_POSITIONS[anchor_id]
    nodes['phone_0'] = None  # Unknown position
    
    # Run PGO
    solver = PGOSolver()
    try:
        result = solver.solve(nodes, all_edges, DEFAULT_ANCHOR_POSITIONS)
        if result.success and 'phone_0' in result.node_positions:
            return result.node_positions['phone_0'][:2]  # Return only x, y (ignore z)
    except Exception as e:
        # PGO failed
        pass
    
    return None

In [7]:

def estimate_positions_single_anchor(measurements: Dict[str, List[List[float]]], 
                                   anchor_id: int,
                                   ground_truth: Tuple[float, float]) -> List[np.ndarray]:
    """
    Estimate position using a single anchor with bin averaging.
    For 1-anchor case, we first take the average of filtered measurements in the bin,
    then transform to global coordinates and use the actual measured position.
    This matches the approach used for multi-anchor PGO cases.
    """
    anchor_id_str = str(anchor_id)
    if anchor_id_str not in measurements:
        return []
    
    vectors = measurements[anchor_id_str]
    if not vectors:
        return []
    
    # First: Calculate the mean of the bin in local coordinates (same as multi-anchor approach)
    local_mean = np.mean(vectors, axis=0)
    
    # Then: Transform the bin mean to global coordinates
    try:
        _, _, global_mean_vector = create_relative_measurement(anchor_id, 0, local_mean)
    except ValueError:
        return []
    
    # Use actual measured position (anchor position + measurement vector)
    anchor_pos_2d = DEFAULT_ANCHOR_POSITIONS[anchor_id][:2]
    
    # Create single position estimate using the bin mean
    estimated_pos = anchor_pos_2d + global_mean_vector[:2]  # Only use x,y components
    
    # Return as single-element list to maintain consistency with function signature
    return [estimated_pos]

In [8]:

def find_worst_case_combinations_with_masking(data_group: List[Dict], 
                                            ground_truth: Tuple[float, float]) -> Dict[int, List[int]]:
    """Find worst-case anchor combinations using measurement masking."""
    
    # Get all available anchors from the data
    all_available_anchors = set()
    for row in data_group:
        available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
        all_available_anchors.update(available_anchors)
    
    all_anchor_ids = sorted(list(all_available_anchors))
    print(f"    Available anchors for this position: {all_anchor_ids}")
    
    worst_combinations = {}
    
    # Handle 1 anchor case separately (simple distance-based estimation)
    if len(all_anchor_ids) >= 1:
        worst_distance_1 = -1
        worst_anchor_1 = None
        
        for anchor_id in all_anchor_ids:
            distances = []
            for row in data_group:
                available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
                if anchor_id not in available_anchors:
                    continue
                
                pos_estimates = estimate_positions_single_anchor(
                    row['filtered_data']['measurements'], 
                    anchor_id, 
                    ground_truth
                )
                
                for pos_estimate in pos_estimates:
                    distance = np.sqrt((pos_estimate[0] - ground_truth[0])**2 + 
                                     (pos_estimate[1] - ground_truth[1])**2)
                    distances.append(distance)
            
            if distances:
                avg_distance = np.mean(distances)
                if avg_distance > worst_distance_1:
                    worst_distance_1 = avg_distance
                    worst_anchor_1 = anchor_id
        
        if worst_anchor_1 is not None:
            worst_combinations[1] = [worst_anchor_1]
            print(f"    1 anchor - worst case: [{worst_anchor_1}] (avg error: {worst_distance_1:.1f}cm)")
    
    # Handle 2-4 anchor cases with PGO and masking
    for num_anchors in range(2, 5):
        if num_anchors > len(all_anchor_ids):
            continue
            
        worst_distance = -1
        worst_combination = None
        
        # Test all combinations of num_anchors
        for anchor_combination in combinations(all_anchor_ids, num_anchors):
            anchor_combination = list(anchor_combination)
            
            # Run PGO for all measurements in this group with this anchor combination
            distances = []
            successful_runs = 0
            
            for row in data_group:
                # Check if this row has measurements from all required anchors
                available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
                if not all(anchor_id in available_anchors for anchor_id in anchor_combination):
                    continue
                
                # Run PGO with masked measurements
                pgo_result = run_pgo_with_masked_measurements(
                    row['filtered_data']['measurements'], 
                    anchor_combination
                )
                
                if pgo_result is not None:
                    # Calculate distance from ground truth (ignore z)
                    distance = np.sqrt((pgo_result[0] - ground_truth[0])**2 + 
                                     (pgo_result[1] - ground_truth[1])**2)
                    distances.append(distance)
                    successful_runs += 1
            
            if distances:
                avg_distance = np.mean(distances)
                # For worst case selection: choose the combination with highest average error
                if avg_distance > worst_distance:
                    worst_distance = avg_distance
                    worst_combination = anchor_combination
        
        if worst_combination is not None:
            worst_combinations[num_anchors] = worst_combination
            print(f"    {num_anchors} anchors - worst case: {worst_combination} (avg error: {worst_distance:.1f}cm)")
        else:
            print(f"    Warning: No valid combination found for {num_anchors} anchors")
    
    return worst_combinations

In [9]:

def calculate_position_statistics_with_masking(data_group: List[Dict], 
                                             anchor_combinations: Dict[int, List[int]],
                                             ground_truth: Tuple[float, float]) -> Dict[int, Tuple[float, float, float, float]]:
    """Calculate position statistics for each anchor combination using masking."""
    
    statistics = {}
    
    for num_anchors, anchor_ids in anchor_combinations.items():
        positions = []
        
        if num_anchors == 1:
            # Handle single anchor case
            anchor_id = anchor_ids[0]
            for row in data_group:
                available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
                if anchor_id not in available_anchors:
                    continue
                
                pos_estimates = estimate_positions_single_anchor(
                    row['filtered_data']['measurements'], 
                    anchor_id, 
                    ground_truth
                )
                
                positions.extend(pos_estimates)
        else:
            # Handle multi-anchor case with PGO and masking
            for row in data_group:
                # Check if this row has measurements from all required anchors
                available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
                if not all(anchor_id in available_anchors for anchor_id in anchor_ids):
                    continue
                    
                # Run PGO with masked measurements
                pgo_result = run_pgo_with_masked_measurements(
                    row['filtered_data']['measurements'], 
                    anchor_ids
                )
                
                if pgo_result is not None:
                    positions.append(pgo_result)
        
        if positions:
            positions_array = np.array(positions)
            mean_x = np.mean(positions_array[:, 0])
            mean_y = np.mean(positions_array[:, 1])
            std_x = np.std(positions_array[:, 0])
            std_y = np.std(positions_array[:, 1])
            
            statistics[num_anchors] = (mean_x, mean_y, std_x, std_y)
            print(f"    {num_anchors} anchors: {len(positions)} successful position estimates")
        else:
            print(f"    Warning: No valid position estimates for {num_anchors} anchors")
    
    return statistics

In [35]:
def create_god_plot_v5(orientation: str, data: List[Dict], output_dir: str):
    """Create a god plot for a specific orientation with proper measurement masking."""

    # Group data by ground truth position
    position_groups = defaultdict(list)
    for row in data:
        if row['orientation'] == orientation:
            pos_key = (row['ground_truth_x'], row['ground_truth_y'])
            position_groups[pos_key].append(row)

    if not position_groups:
        print(f"Warning: No data found for orientation {orientation}")
        return

    print(f"\nProcessing orientation {orientation}:")
    print(f"Found {len(position_groups)} positions: {list(position_groups.keys())}")

    # Colors for different anchor counts (worst to best) - Red to Green progression
    plot_colors = {1: '#FF1A1A', 2: '#FF6600', 3: '#FFCC00', 4: '#00CC44'}
    # Pastel colors for labels (softer, less intrusive)
    label_colors = {1: '#FFB3B3', 2: '#FFD1A3', 3: '#FFF4A3', 4: '#A3E6A3'}
    # Create list of colors for the anchors
    anchor_colors = ['red','green','blue','magenta']

    # Process each position and create a separate figure per position
    for pos_key, data_group in position_groups.items():
        ground_truth = pos_key
        print(f"\n  Processing position {ground_truth}:")

        # Create a new figure for this position
        fig, ax = plt.subplots(figsize=(12, 10))

        # Plot anchor positions (on every per-position figure)
        for anchor_id, pos in DEFAULT_ANCHOR_POSITIONS.items():
            ax.plot(pos[0], pos[1], 's', markersize=12, color=anchor_colors[anchor_id],
                    label='Anchor' if anchor_id == 0 else '', zorder=10)
            ax.annotate(f'A{anchor_id}', (pos[0], pos[1]), xytext=(6, 6),
                       textcoords='offset points', fontsize=10, color=anchor_colors[anchor_id],
                       fontweight='bold', zorder=11)

        # Plot ground truth position
        ax.plot(ground_truth[0], ground_truth[1], 'ko', markersize=8,
                label='Ground Truth', zorder=9)
        ax.annotate(f'GT({ground_truth[0]:.0f},{ground_truth[1]:.0f})',
                   (ground_truth[0], ground_truth[1]), xytext=(10, -20),
                   textcoords='offset points', fontsize=10,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.9),
                   zorder=9)

        # Find worst-case anchor combinations using masking
        worst_combinations = find_worst_case_combinations_with_masking(data_group, ground_truth)

        # Calculate statistics for each anchor combination
        statistics = calculate_position_statistics_with_masking(data_group, worst_combinations, ground_truth)

        # Plot position estimates with error bars for each anchor count
        for num_anchors in sorted(statistics.keys()):
            mean_x, mean_y, std_x, std_y = statistics[num_anchors]

            plot_color = plot_colors.get(num_anchors, 'gray')
            label_color = label_colors.get(num_anchors, 'gray')

            ax.errorbar(mean_x, mean_y, xerr=std_x, yerr=std_y,
                       fmt='o', markersize=6, markeredgecolor='k', markerfacecolor=plot_color,
                       color=plot_color, capsize=4, capthick=1, elinewidth=1.0,
                       label=f'{num_anchors} Anchor{"s" if num_anchors > 1 else ""} (Worst Case)',
                       zorder=7)

            distance_error = np.sqrt((mean_x - ground_truth[0])**2 + (mean_y - ground_truth[1])**2)
            offsets = {1: (20, 20), 2: (-60, 20), 3: (20, -40), 4: (-60, -40)}
            offset_x, offset_y = offsets.get(num_anchors, (15, 15))

            anchor_text = f"Err: {distance_error:.1f}cm"
            ax.annotate(anchor_text, (mean_x, mean_y),
                       xytext=(offset_x, offset_y), textcoords='offset points', fontsize=9,
                       bbox=dict(boxstyle='round,pad=0.3', facecolor=label_color, alpha=0.85),
                       zorder=6)

        # Per-position summary text
        summary_lines = [f"Orientation {orientation} - Position ({int(ground_truth[0])},{int(ground_truth[1])})"]
        for num_anchors in sorted(plot_colors.keys()):
            if num_anchors in statistics:
                mean_x, mean_y, std_x, std_y = statistics[num_anchors]
                dist_err = np.sqrt((mean_x - ground_truth[0])**2 + (mean_y - ground_truth[1])**2)
                summary_lines.append(f"{num_anchors} Anchors: {dist_err:.1f}cm (std_x={std_x:.1f}, std_y={std_y:.1f})")

        summary_text = "\n".join(summary_lines)
        ax.text(0.02, 0.98, summary_text, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.9))

        # Formatting
        ax.set_xlabel('X Position (cm)', fontsize=12)
        ax.set_ylabel('Y Position (cm)', fontsize=12)
        ax.set_title(f'God Plot - Orientation {orientation} - Position ({int(ground_truth[0])},{int(ground_truth[1])})', fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='upper right', fontsize=10)
        ax.set_aspect('equal')
        ax.set_xlim(-100, 700)
        ax.set_ylim(-100, 700)

        # Save per-position plot
        x_int = int(ground_truth[0])
        y_int = int(ground_truth[1])
        output_path = os.path.join(output_dir, f'god_plot_v5_orientation_{orientation}_pos_{x_int}_{y_int}.png')
        plt.tight_layout()
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close(fig)

        print(f"Saved god plot v5 for orientation {orientation}, position ({x_int},{y_int}) to {output_path}")

In [11]:

def create_detailed_comparison_table(data: List[Dict], output_dir: str):
    """Create detailed PGO vs Single Anchor comparison table matching the format from pgo_vs_single_anchor_analysis.py"""
    
    # Group data by ground truth position and orientation
    position_groups = defaultdict(list)
    for row in data:
        pos_key = (row['ground_truth_x'], row['ground_truth_y'], row['orientation'])
        position_groups[pos_key].append(row)
    
    # Collect results for each position/orientation group
    table_results = []
    
    for pos_key, data_group in position_groups.items():
        ground_truth_x, ground_truth_y, orientation = pos_key
        ground_truth = (ground_truth_x, ground_truth_y)
        
        # Get PGO results (from CSV)
        pgo_positions = []
        pgo_errors = []
        for row in data_group:
            pgo_pos = np.array([row['pgo_x'], row['pgo_y']])
            pgo_positions.append(pgo_pos)
            gt_pos = np.array([ground_truth_x, ground_truth_y])
            pgo_error = np.linalg.norm(pgo_pos - gt_pos)
            pgo_errors.append(pgo_error)
        
        # Find worst-case single anchor combinations
        worst_combinations = find_worst_case_combinations_with_masking(data_group, ground_truth)
        
        # Get single anchor results (worst case for 1 anchor)
        worst_anchor_errors = []
        worst_anchor_id = None
        
        if 1 in worst_combinations:
            worst_anchor_id = worst_combinations[1][0]
            
            for row in data_group:
                available_anchors = list(map(int, row['filtered_data']['measurements'].keys()))
                if worst_anchor_id not in available_anchors:
                    continue
                
                pos_estimates = estimate_positions_single_anchor(
                    row['filtered_data']['measurements'], 
                    worst_anchor_id, 
                    ground_truth
                )
                
                for pos_estimate in pos_estimates:
                    error = np.sqrt((pos_estimate[0] - ground_truth_x)**2 + 
                                  (pos_estimate[1] - ground_truth_y)**2)
                    worst_anchor_errors.append(error)
        
        # Calculate statistics
        mean_pgo_x = np.mean([pos[0] for pos in pgo_positions])
        mean_pgo_y = np.mean([pos[1] for pos in pgo_positions])
        mean_pgo_error = np.mean(pgo_errors)
        pgo_stderr = np.std(pgo_errors) / np.sqrt(len(pgo_errors)) if len(pgo_errors) > 1 else 0.0
        
        if worst_anchor_errors:
            mean_worst_error = np.mean(worst_anchor_errors)
            worst_stderr = np.std(worst_anchor_errors) / np.sqrt(len(worst_anchor_errors)) if len(worst_anchor_errors) > 1 else 0.0
        else:
            mean_worst_error = 0.0
            worst_stderr = 0.0
            worst_anchor_id = 0
        
        table_results.append({
            'ground_truth_x': int(ground_truth_x),
            'ground_truth_y': int(ground_truth_y),
            'orientation': orientation,
            'count': len(data_group),
            'mean_pgo_x': mean_pgo_x,
            'mean_pgo_y': mean_pgo_y,
            'mean_pgo_error': mean_pgo_error,
            'pgo_stderr': pgo_stderr,
            'worst_anchor_id': worst_anchor_id if worst_anchor_id is not None else 0,
            'mean_worst_error': mean_worst_error,
            'worst_stderr': worst_stderr
        })
    
    # Sort results for consistent output
    table_results.sort(key=lambda x: (x['ground_truth_x'], x['ground_truth_y'], x['orientation']))
    
    # Create the formatted table
    table_lines = []
    table_lines.append("")
    table_lines.append("=" * 120)
    table_lines.append("PGO vs SINGLE ANCHOR ACCURACY ANALYSIS RESULTS (2D - X,Y only)")
    table_lines.append("Generated from God Plot Script with Realistic Bin Averaging")
    table_lines.append("=" * 120)
    table_lines.append("Position     PGO X    PGO Y    Orient   Count  PGO Error    PGO StdErr   Worst Anchor Worst Error  Worst StdErr")
    table_lines.append("-" * 120)
    
    for result in table_results:
        row = f"({result['ground_truth_x']:6d},{result['ground_truth_y']:6d},{result['orientation']}) {result['mean_pgo_x']:6.1f}    {result['mean_pgo_y']:6.1f}    {result['orientation']}        {result['count']}      {result['mean_pgo_error']:6.1f}        {result['pgo_stderr']:4.1f}          {result['worst_anchor_id']}            {result['mean_worst_error']:6.1f}        {result['worst_stderr']:4.1f}"
        table_lines.append(row)
    
    table_lines.append("=" * 120)
    table_lines.append("")
    
    # Save table
    table_content = "\n".join(table_lines)
    output_path = os.path.join(output_dir, 'pgo_vs_single_anchor_detailed_table_from_godplot.txt')
    with open(output_path, 'w') as f:
        f.write(table_content)
    
    print(f"Detailed comparison table saved to {output_path}")
    print(table_content)
    
    return table_content

In [24]:
"""Main function to generate all god plots with proper measurement masking."""

# Paths
csv_path = '/Users/shihcheng/Documents/Work/Python/uwb-localization-mesh/Data_collection/Data/28oct/datapoints28oct.csv'
output_dir = '/Users/shihcheng/Documents/Work/Python/uwb-localization-mesh/Data_collection/Data/28oct/god_plots'

print("Loading data...")
data = load_data(csv_path)
print(f"Loaded {len(data)} data points")

Loading data...
Loaded 46 data points


In [25]:
# Get unique orientations
orientations = sorted(set(row['orientation'] for row in data))
print(f"Found orientations: {orientations}")

Found orientations: ['A', 'B', 'C', 'U']


In [23]:
orientation = orientations[0]
position_groups = defaultdict(list)
for row in data:
    if row['orientation'] == orientation:
        pos_key = (row['ground_truth_x'], row['ground_truth_y'])
        position_groups[pos_key].append(row)
print(position_groups)    
print(len(position_groups))

for pg in position_groups:
    fig, ax = plt.subplots(figsize=(18, 14))
    for anchor_id, pos in DEFAULT_ANCHOR_POSITIONS.items():
        ax.plot(pos[0], pos[1], 's', markersize=15, color='red', 
                label='Anchor' if anchor_id == 0 else '', zorder=10)
        ax.annotate(f'A{anchor_id}', (pos[0], pos[1]), xytext=(8, 8), 
                   textcoords='offset points', fontsize=12, color='red', 
                   fontweight='bold', zorder=11)


defaultdict(<class 'list'>, {(0.0, 0.0): [{'timestamp': '1761603523.269979', 'ground_truth_x': 0.0, 'ground_truth_y': 0.0, 'ground_truth_z': 0.0, 'pgo_x': 6.76665770279781, 'pgo_y': -6.35605455742522, 'pgo_z': -161.67530101677747, 'orientation': 'A', 'filtered_binned_data_json': '{"bin_start_time": 1761632322.262254, "bin_end_time": 1761632323.262254, "phone_node_id": 0, "measurements": {"1": [[469.90937065730844, -414.57244896890165, 144.096731137038], [483.0202378558234, -402.4372901463186, 134.85057404917302], [487.71772289866595, -399.4767096605781, 131.4754015369348], [463.5562874362399, -402.2536556172653, 195.0475965524225], [480.269884041355, -409.60970583283404, 142.02016536549488]], "3": [[149.33227822759216, -12.566029328178901, -182.31282342797599], [157.68913012274092, -14.933779797252656, -168.14315466917006], [158.54754694598273, -25.906332117865066, -172.88012411382135], [155.2193447963732, -16.75258284366866, -174.30520924243956], [167.27139413474083, -15.2817635853131

In [36]:
  # Generate god plot for each orientation
for orientation in orientations:
    print(f"\n{'='*60}")
    print(f"Generating god plot v5 for orientation {orientation}...")
    print(f"{'='*60}")
    create_god_plot_v5(orientation, data, output_dir)


Generating god plot v5 for orientation A...

Processing orientation A:
Found 4 positions: [(0.0, 0.0), (120.0, 180.0), (240.0, 0.0), (240.0, 300.0)]

  Processing position (0.0, 0.0):
    Available anchors for this position: [0, 1, 2, 3]
    1 anchor - worst case: [3] (avg error: 80.6cm)
    2 anchors - worst case: [1, 3] (avg error: 55.1cm)
    3 anchors - worst case: [1, 2, 3] (avg error: 44.5cm)
    4 anchors - worst case: [0, 1, 2, 3] (avg error: 18.0cm)
    1 anchors: 3 successful position estimates
    2 anchors: 3 successful position estimates
    3 anchors: 3 successful position estimates
    4 anchors: 1 successful position estimates
Saved god plot v5 for orientation A, position (0,0) to /Users/shihcheng/Documents/Work/Python/uwb-localization-mesh/Data_collection/Data/28oct/god_plots/god_plot_v5_orientation_A_pos_0_0.png

  Processing position (120.0, 180.0):
    Available anchors for this position: [0, 1, 2, 3]
    1 anchor - worst case: [0] (avg error: 107.6cm)
    2 anchor

In [ ]:
print(f"\n{'='*60}")
print(f"All god plots v5 generated successfully in {output_dir}")
print(f"{'='*60}")

# Generate detailed comparison table
print(f"\n{'='*60}")
print(f"Generating detailed comparison table...")
print(f"{'='*60}")
create_detailed_comparison_table(data, output_dir)